# Environment Setup

In [2]:
#r "nuget:Microsoft.DotNet.Interactive.SqlServer,*-*"

Installed Packages Microsoft.DotNet.Interactive.SqlServer, 1.0.0-beta.23258.3

Loading extensions from `/Users/Steven.Tong/.nuget/packages/microsoft.dotnet.interactive.sqlserver/1.0.0-beta.23258.3/interactive-extensions/dotnet/Microsoft.DotNet.Interactive.SqlServer.dll`

Query Microsoft SQL Server databases. 
 This extension adds support for connecting to Microsoft SQL Server databases using the #!connect mssql magic command. For more information, run a cell using the #!sql magic command.

In [3]:
#!connect mssql --kernel-name AGDemo-1a "Persist Security Info=False; TrustServerCertificate=True; Integrated Security=true; Initial Catalog=master; Server=rp-sql19ags-1a.perf.rubrik.com;"
#!connect mssql --kernel-name AGDemo-1b "Persist Security Info=False; TrustServerCertificate=True; Integrated Security=true; Initial Catalog=master; Server=rp-sql19ags-1b.perf.rubrik.com;"
#!connect mssql --kernel-name AGDemo-1c "Persist Security Info=False; TrustServerCertificate=True; Integrated Security=true; Initial Catalog=master; Server=rp-sql19ags-1c.perf.rubrik.com;"

Kernel added: #!sql-AGDemo-1a

Kernel added: #!sql-AGDemo-1b

Kernel added: #!sql-AGDemo-1c

In [4]:
# Rubrik Connection Information
$ServiceAccountPath = '/Users/Steven.Tong/OneDrive - Rubrik Inc/git/rsc-service-account-rr.json'
$ServiceAccountContent = Get-Content -Path "$serviceAccountPath" -ErrorAction Stop | ConvertFrom-Json

$Server.access_token_uri
$ServiceAccountID = $ServiceAccountContent.client_id
$Secret = $ServiceAccountContent.client_secret

# Primary SQL Server
$PrimarySQLServerInstance = 'rp-sql19ags-1c.perf.rubrik.com'

# Secondary SQL Servers
$SecondarySQLServerInstance = 'rp-sql19ags-1a.perf.rubrik.com', 'rp-sql19ags-1b.perf.rubrik.com'

# Availability Group Name
$AvailabilityGroupName = 'rp-sql19ags-g1'
# Source Instance Name
$Instance = "MSSQLSERVER"
# Source Database Name
$DatabaseName = 'TongExport'

# Connect to Rubrik

In [5]:
Connect-RubrikSecurityCloud -ServiceAccountPath $ServiceAccountPath

Connected!


# Refresh the Hosts of the SQL Servers in RSC

In [6]:
$QueryParms = @{
    "hostname" = "$($PrimarySQLServerInstance)"
    "instance" = $Instance
}
$RSCMSQLInstance = Invoke-RubrikQuery -Path ./GetMSSQLInstance.gql -QueryParams $QueryParms


$QueryParms = @{
    "id" = "$($RSCMSQLInstance.id)"
}
 Invoke-RubrikQuery -Path ./PhysicalHostRefreshMutation.gql -QueryParams $QueryParms

foreach ($SQLInstance in $SecondarySQLServerInstance){
    $QueryParms = @{
        "hostname" = "$($SQLInstance)"
        "instance" = $Instance
    }
    $RSCMSQLInstance = Invoke-RubrikQuery -Path ./GetMSSQLInstance.gql -QueryParams $QueryParms

    $QueryParms = @{
        "id" = "$($RSCMSQLInstance.id)"
    }
    Invoke-RubrikQuery -Path ./PhysicalHostRefreshMutation.gql -QueryParams $QueryParms
}
Start-Sleep -Seconds 45

# Remove Database from Availability Group

In [7]:
Remove-DbaAgDatabase -SQLInstance $PrimarySQLServerInstance -Database $DatabaseName -Confirm:$false -Verbose

VERBOSE: [16:57:13][Connect-DbaInstance] String is passed in, will build server object from instance object and other parameters, do some checks and then return the server object
VERBOSE: [16:57:13][Connect-DbaInstance] authentication method is 'local integrated'
VERBOSE: Performing the operation "Removing availability group database [TongExport]" on target "rp-sql19ags-1c.perf.rubrik.com".

ComputerName      : rp-sql19ags-1c
InstanceName      : MSSQLSERVER
SqlInstance       : rp-sql19ags-1c
AvailabilityGroup : rp-sql19ags-g1
Database          : TongExport
Status            : Removed




## Remove Database from the SQL Server Instances

In [9]:
foreach ($SQLInstance in $SecondarySQLServerInstance){
    Remove-DbaDatabase -SqlInstance $SQLInstance -Database $DatabaseName -Confirm:$false -Verbose
}

Remove-DbaDatabase -SqlInstance $PrimarySQLServerInstance -Database $DatabaseName -Confirm:$false -Verbose

VERBOSE: [16:57:40][Connect-DbaInstance] String is passed in, will build server object from instance object and other parameters, do some checks and then return the server object
VERBOSE: [16:57:40][Connect-DbaInstance] authentication method is 'local integrated'
VERBOSE: [16:57:40][Connect-DbaInstance] String is passed in, will build server object from instance object and other parameters, do some checks and then return the server object
VERBOSE: [16:57:40][Connect-DbaInstance] authentication method is 'local integrated'
VERBOSE: [16:57:41][Connect-DbaInstance] String is passed in, will build server object from instance object and other parameters, do some checks and then return the server object
VERBOSE: [16:57:41][Connect-DbaInstance] authentication method is 'local integrated'


## Refresh the Hosts of the SQL Servers in RSC

In [8]:
$QueryParms = @{
    "hostname" = "$($PrimarySQLServerInstance)"
    "instance" = $Instance
}
$RSCMSQLInstance = Invoke-RubrikQuery -Path ./GetMSSQLInstance.gql -QueryParams $QueryParms


$QueryParms = @{
    "id" = "$($RSCMSQLInstance.id)"
}
 Invoke-RubrikQuery -Path ./PhysicalHostRefreshMutation.gql -QueryParams $QueryParms

foreach ($SQLInstance in $SecondarySQLServerInstance){
    $QueryParms = @{
        "hostname" = "$($SQLInstance)"
        "instance" = $Instance
    }
    $RSCMSQLInstance = Invoke-RubrikQuery -Path ./GetMSSQLInstance.gql -QueryParams $QueryParms

    $QueryParms = @{
        "id" = "$($RSCMSQLInstance.id)"
    }
    Invoke-RubrikQuery -Path ./PhysicalHostRefreshMutation.gql -QueryParams $QueryParms
}
Start-Sleep -Seconds 45

# Get the ID of the Availability Group

In [6]:
$QueryParms = @{
    "AvailabilityGroupName" = $($AvailabilityGroupName)
}
$RSCMSSQLAvailabilityGroup = Invoke-RubrikQuery -Path ./GetMSSQLAvailabilityGroup.gql -QueryParams $QueryParms

if ($RSCMSSQLAvailabilityGroup.id -eq $null) {
    Write-Error "No AG found with name: $AvailabilityGroupName"
}
$RSCMSSQLAvailabilityGroup


id                                   name
--                                   ----
37d606ef-0e98-5e15-bb93-b50497484694 rp-sql19ags-g1



# Get the ID of the Database

In [ ]:
$QueryParms = @{
    "fid" = "$($RSCMSSQLAvailabilityGroup.id)"
    "DatabaseName" = "$($DatabaseName)"
}
$RSCMSSQLDatabase = Invoke-RubrikQuery -Path ./GetMSSQLDatabaseFromAvailabilityGroup.gql -QueryParams $QueryParms

if ($RSCMSSQLDatabase.logicalChildConnection.nodes.count -eq 0) {
    Write-Error "No database found with name: $DatabaseName, on AG: $AvailabilityGroupName"
}
$RSCMSSQLDatabase.logicalChildConnection.nodes

# Get the Latest Recovery Point

In [96]:
$QueryParms = @{
    "id" = "$($RSCMSSQLDatabase.logicalChildConnection.nodes[0].id)"
}
$RSCMSSQLRecoverableRange = Invoke-RubrikQuery -Path ./MssqlDatabaseDetailsRecoverableRangesQuery.gql -QueryParams $QueryParms | Sort-Object {$_.data.endTime}
$RSCMSSQLDatabaseLatestRecoveryPoint = ($RSCMSSQLRecoverableRange.data.endTime[-1]).ToUniversalTime().ToString('yyyy-MM-ddTHH:mm:ss.fffZ')
$RSCMSSQLDatabaseLatestRecoveryPoint

2023-05-11T23:03:11.000Z


# Get the Original DB File Locations

In [97]:
$QueryParms = @{
    "input" = @{
        "id" = "$($RSCMSSQLDatabase.logicalChildConnection.nodes[0].id)"
        "time" = $($RSCMSSQLDatabaseLatestRecoveryPoint)
    }
}
$RSCMSSQLFileLocations = Invoke-RubrikQuery -Path ./AllMssqlDatabaseRestoreFilesQuery.gql -QueryParams $QueryParms

$TargetFilePaths = @()

foreach ($DBfile in $RSCMSSQLFileLocations.items) {
    $originalName = [system.io.fileinfo]$DBFile.originalName
    $newFilename = $originalName.Basename + $originalName.Extension
    $targetFilePath = @{
        "newFilename" = $newFilename
        "exportPath" = $DBfile.originalPath
        "logicalName" = $DBFile.logicalName
    }
    $TargetFilePaths += $targetFilePath
}

$targetFilePaths


Name                           Value
----                           -----
exportPath                     c:\mnt\sqldata
logicalName                    AdventureWorks2017
newFilename                    ProductionDatabase_tongexport.mdf
exportPath                     c:\mnt\sqllogs
logicalName                    AdventureWorks2017_log
newFilename                    ProductionDatabase_Log_tongexport.ldf



# Export the Database to the Secondary Replicas

In [98]:
foreach ($SQLInstance in $SecondarySQLServerInstance){
    $QueryParms = @{
        "hostname" = "$($SQLInstance)"
        "instance" = "$($Instance)"
    }
    $RSCMSQLInstance = Invoke-RubrikQuery -Path ./GetMSSQLInstance.gql -QueryParams $QueryParms
    $RSCMSQLInstance    

    $QueryParms = @{
        "input" = @{
            "id" = "$($RSCMSSQLDatabase.logicalChildConnection.nodes[0].id)"
            "config" = @{
                "recoveryPoint" = @{
                    "date" = "$($RSCMSSQLDatabaseLatestRecoveryPoint)"
                }
                "targetInstanceId" = "$($RSCMSQLInstance.physicalChildConnection.nodes.id)"
                "targetDatabaseName" = "$($DatabaseName)"
                "targetDataFilePath" = ""
                "targetLogFilePath" = ""
                "targetFilePaths" = $TargetFilePaths
                "allowOverwrite" = $true
                "finishRecovery" = $false
            }
        }
    }
    Invoke-RubrikQuery -Path ./MssqlDatabaseExportMutation.gql -QueryParams $QueryParms
}
Start-Sleep -Seconds 45


id                                   name                           cbtStatus physicalChildConnecti
                                                                              on
--                                   ----                           --------- ---------------------
1176c065-3434-5c87-b69f-597560c8cb17 rp-sql19ags-1a.perf.rubrik.com Enabled   @{nodes=System.Objec…
6e02668f-2d63-5a2d-90bf-204df7676728 rp-sql19ags-1b.perf.rubrik.com Enabled   @{nodes=System.Objec…



# Export the Database to the Primary Replica

In [99]:
$QueryParms = @{
    "hostname" = "$($PrimarySQLServerInstance)"
    "instance" = "$($Instance)"
}
$RSCMSQLInstance = Invoke-RubrikQuery -Path ./GetMSSQLInstance.gql -QueryParams $QueryParms
$RSCMSQLInstance    

$QueryParms = @{
    "input" = @{
        "id" = "$($RSCMSSQLDatabase.logicalChildConnection.nodes[0].id)"
        "config" = @{
            "recoveryPoint" = @{"date" = "$($RSCMSSQLDatabaseLatestRecoveryPoint)"}
            "targetInstanceId" = "$($RSCMSQLInstance.physicalChildConnection.nodes.id)"
            "targetDatabaseName" = "$($DatabaseName)"
            "targetDataFilePath" = ""
            "targetLogFilePath" = ""
            "targetFilePaths" = $TargetFilePaths
            "allowOverwrite" = $true
            "finishRecovery" = $true
        }
    }
}
Invoke-RubrikQuery -Path ./MssqlDatabaseExportMutation.gql -QueryParams $QueryParms
Start-Sleep -Seconds 45


id                                   name                           cbtStatus physicalChildConnecti
                                                                              on
--                                   ----                           --------- ---------------------
23fdb155-57bb-5cbb-9ec6-847bad6f09aa rp-sql19ags-1c.perf.rubrik.com Enabled   @{nodes=System.Objec…



# Add the Database into the AG on the Primary Replica

In [2]:
$Query = "ALTER AVAILABILITY GROUP [$($AvailabilityGroupName)] ADD DATABASE [$($DatabaseName)];"
Invoke-DbaQuery -SqlInstance $PrimarySQLServerInstance -Query $Query

# Add the Database into the AG on Secondary Replicas

In [3]:
foreach ($SQLInstance in $SecondarySQLServerInstance){
    $Query = "ALTER DATABASE [$($DatabaseName)] SET HADR AVAILABILITY GROUP = [$($AvailabilityGroupName)];"
    Invoke-DbaQuery -SqlInstance $SQLInstance -Query $Query
}